In [6]:
#NEW MODEL

#ROLLING (absolute) MEAN OF NORMALIZED DATA

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
import numpy as np
import os

input_dir = "mindrove_dataset"
dataval= pd.read_csv(os.path.join(input_dir, "definitivo_dataset.csv"))

emgval = np.abs(dataval.iloc[:,2:10] * 0.045)
data = np.array(emgval)

#FIRST: NORMALIZE
normalized_data = np.empty_like(data)
scalers = []
for i in range(data.shape[1]):
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(data[:, i].reshape(-1, 1))
    scalers.append(scaler)
for i, scaler in enumerate(scalers):
    normalized_data[:, i] = scaler.transform(data[:, i].reshape(-1, 1)).flatten()

taskval = dataval.iloc[:,1]
newdata = np.zeros(((data.shape[0]),9))
newdata[:, 0] = taskval
newdata[:, 1:] = normalized_data

#SECOND: ROLLING MEAN FOR EACH COLUMN
newdata = pd.DataFrame(newdata) #convert to dataframe
roll = newdata.rolling(window=10, step=1).mean()
roll2 = np.zeros(roll.shape)

repetitions = [1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6,7,7,7]
count=-1
#he establecido un treshold para que asigne un task u otro y no haya decimales y
#que en total solo sean 7 tasks

for i in range(1,(len(roll))):
        if not roll.iloc[i,0] == 0 and roll.iloc[i-1,0] == 0:
            count += 1
        if not roll.iloc[i,0] == 0 and roll.iloc[i,0] < (repetitions[count]/2):
            roll2[i,0] = 0
        elif not roll.iloc[i,0] == 0 and roll.iloc[i,0] >= (repetitions[count]/2):
            roll2[i,0] = repetitions[count]
#eliminate first 2 values NaN
roll2[9:,1:]= roll.iloc[9:,1:]
MAV = roll2[9:,:]


In [7]:
#ROLLING SSI OF NORMALIZED DATA

#FIRST: SQUARED THE NORMALIZED DATA
normsquared = np.square(normalized_data)   
taskval = dataval.iloc[:,1]
newdata = np.zeros(((data.shape[0]),9))
newdata[:, 0] = taskval
newdata[:, 1:] = normsquared

#SECOND: ROLLING SSI FOR EACH COLUMN
import pandas as pd
newdata = pd.DataFrame(newdata) #convert to dataframe
roll = newdata.rolling(window=10, step=1).mean()
roll2 = np.zeros(roll.shape)
repetitions = [1,1,1,2,2,2,3,3,3,4,4,4,5,5,5,6,6,6,7,7,7]
count=-1
#he establecido un treshold para que asigne un task u otro y no haya decimales y
#que en total solo sean 7 tasks

for i in range(1,(len(roll))):
        if not roll.iloc[i,0] == 0 and roll.iloc[i-1,0] == 0:
            count += 1
        if not roll.iloc[i,0] == 0 and roll.iloc[i,0] < (repetitions[count]/2):
            roll2[i,0] = 0
        elif not roll.iloc[i,0] == 0 and roll.iloc[i,0] >= (repetitions[count]/2):
            roll2[i,0] = repetitions[count]
#eliminate first 9 values NaN           
roll2[9:,1:]= roll.iloc[9:,1:]
SSI = roll2[9:,:]

In [9]:
#JOIN
datasvm = pd.concat([MAV, SSI[:,1:]], axis=1)

TypeError: cannot concatenate object of type '<class 'numpy.ndarray'>'; only Series and DataFrame objs are valid